# Figure S2 : DM step response

Response to adding glucose in healthy and DM cells.

In the diabetic parameter set. We adjusted

- PDH capacity
- ETC capacity
- F1 synthase capacity
- Proton leak rate

In [ ]:
using DifferentialEquations
using ModelingToolkit
using MitochondrialDynamics
using MitochondrialDynamics: t, Glc, G3P, Pyr, NADH_c, NADH_m, Ca_c, Ca_m, ΔΨm, ATP_c, ADP_c, AMP_c, degavg, x, VmaxPDH, pHleak, VmaxF1, VmaxETC
using MitochondrialDynamics.Utils: second, μM, mV, mM, Hz, minute

using Plots
pyplot()
Plots.pyrcparams["font.size"] = 14
Plots.pyrcparams["font.sans-serif"] = "Arial"
Plots.pyrcparams["font.family"] = "sans-serif"

In [ ]:
function glc_step(t)
    if t < 20minute
        g = 5.0mM
    elseif 20minute <= t < 40minute
        g = 10.0mM
    else
        g = 15.0mM
    end
end

@register_symbolic glc_step(t)

In [ ]:
tend = 60minute
@named sys = make_model(;glcrhs=glc_step(t))
prob = ODEProblem(sys, DEFAULT_U0, tend)

In [ ]:
function make_dm_prob(prob; rPDH=0.5, rETC=0.75, rHL=1.4, rF1=0.5)
    newparams = copy(prob.p)
    idxVmaxPDH = findfirst(isequal(VmaxPDH), parameters(sys))
    idxpHleak = findfirst(isequal(pHleak), parameters(sys))
    idxVmaxF1 = findfirst(isequal(VmaxF1), parameters(sys))
    idxVmaxETC = findfirst(isequal(VmaxETC), parameters(sys))
    newparams[idxVmaxPDH] = prob.p[idxVmaxPDH] * rPDH
    newparams[idxpHleak] = prob.p[idxpHleak] * rHL
    newparams[idxVmaxF1] = prob.p[idxVmaxF1] * rF1
    newparams[idxVmaxETC] = prob.p[idxVmaxETC] * rETC
    return remake(prob, p=newparams)
end

In [ ]:
probDM = make_dm_prob(prob)

In [ ]:
sol = solve(prob, tstops=[20minute, 40minute])
solDM = solve(probDM, tstops=[20minute, 40minute])

In [ ]:
ts = range(0.0, tend, length=300)
g3p = [sol(ts, idxs=G3P).u solDM(ts, idxs=G3P).u] .* 1000
pyr = [sol(ts, idxs=Pyr).u solDM(ts, idxs=Pyr).u] .* 1000
nadh_c = [sol(ts, idxs=NADH_c).u solDM(ts, idxs=NADH_c).u] .* 1000
nadh_m = [sol(ts, idxs=NADH_m).u solDM(ts, idxs=NADH_m).u] .* 1000
ca_c = [sol(ts, idxs=Ca_c).u solDM(ts, idxs=Ca_c).u] .* 1000
ca_m = [sol(ts, idxs=Ca_m).u solDM(ts, idxs=Ca_m).u] .* 1000
dpsi = [sol(ts, idxs=ΔΨm).u solDM(ts, idxs=ΔΨm).u] .* 1000
atp_c = [sol(ts, idxs=ATP_c).u solDM(ts, idxs=ATP_c).u] .* 1000
adp_c = [sol(ts, idxs=ADP_c).u solDM(ts, idxs=ADP_c).u] .* 1000
td = atp_c ./ adp_c
deg = [sol(ts, idxs=degavg).u solDM(ts, idxs=degavg).u]

In [ ]:
labels = ["Baseline" "Diabetic"]
tsm = ts ./ 60
pl1 = plot(tsm, g3p, ylabel="G3P (μM)", title="(A)", titlelocation=:left, label=labels, legend=:right)
pl2 = plot(tsm, pyr, ylabel="Pyruvate (μM)", title="(B)", titlelocation=:left, label=labels, legend=:right)
pl3 = plot(tsm, nadh_c, ylabel="Cytosolic NADH (μM)", title="(C)", titlelocation=:left, label=labels, legend=:right)
pl4 = plot(tsm, nadh_m, ylabel="Mitochondrial NADH (μM)", title="(D)", titlelocation=:left, label=labels, legend=:right)
pl5 = plot(tsm, ca_c, ylabel="Cytosolic Calcium (μM)", title="(E)", titlelocation=:left, label=labels, legend=:right)
pl6 = plot(tsm, ca_m, ylabel="Mitochondrial Calcium (μM)", title="(F)", titlelocation=:left, label=labels, legend=:right)
pl7 = plot(tsm, td, ylabel="ATP/ADP ratio", title="(G)", titlelocation=:left, label=labels, legend=:right)
pl8 = plot(tsm, dpsi, ylabel="Mitochodrial membrane potential (mV)", title="(H)", titlelocation=:left, label=labels, legend=:right)
pl9 = plot(tsm, deg, ylabel="Average Node Degree", title="(I)", titlelocation=:left, label=labels, legend=:right)

plot(pl1, pl2, pl3, pl4, pl5, pl6, pl7, pl8, pl9; size=(2000,2000))

In [ ]:
# Uncomment to generate PDF
# savefig("S1_Fig2.pdf")